In [1]:
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver

def download(method, url, param={}, data={}, timeout=1, maxretries=3):
#     resp = None
    try:
        resp = requests.request(method, url, params=param, data=data, headers={"user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36"})
        resp.raise_for_status() # 에러코드가 발생하면 이벤트 생성 => except로 내려가게 하기 위해서
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            time.sleep(timeout)
            print("재시도", maxretries)
            resp = download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)

    return resp

In [2]:
driver = webdriver.Chrome()

In [3]:
# 검색어
query = '페트병'

# 줌 이미지란
driver.get("http://search.zum.com/search.zum?method=image&option=accu&qm=f_typing&rd=1&query="+query)

In [ ]:
# 이미지 링크 모음
totalLink = []

# 1~10페이지 이미지 링크 모으기
for i in range(1, 10):
    driver.find_elements_by_class_name('gsc-cursor-page')[i].click()
    time.sleep(1)
    totalLink.extend([_.get_attribute('src') for _ in driver.find_elements_by_css_selector('div.gsc-wrapper img.gs-image.gs-image-scalable')])
    time.sleep(1)
driver.close()

In [5]:
count = 1
for _ in totalLink:
    image = download('get', _)
    with open('./images/'+_.split(':')[-1]+'.jpg', 'wb') as f:
        f.write(image.content)
    time.sleep(1)
    print("{0:.2%}".format(count / len(totalLink)))
    count += 1

0.69%
1.39%
2.08%
2.78%
3.47%
4.17%
4.86%
5.56%
6.25%
6.94%
7.64%
8.33%


KeyboardInterrupt: 